# Support Vector Machines 
### with Hyperparameter Tuning

### Hinweis:
Anders als in den Praktika und Vorlesungen verwenden wir hier nicht `Scikit-learn`, sondern die Bibliothek `cuML` (CUDA Machine Learning). Diese ermöglicht es uns, unsere SVMs auf einer NVIDIA GPU zu trainieren, was den Trainingsprozess extrem beschleunigt. An der Herangehensweise und der Art, wie wir die Hyperparameter tunen, ändert sich dadurch jedoch nichts.

1. Laden der aufbereiteten Daten der vorherigen Gruppe: (Da wir cuML verwenden, benutzen wir hier ein cuDF DataFrame anstelle eines Pandas DataFrame.)
2. cuML akzeptiert nur Arrays und keine DataFrames, deswegen müssen wir noch die Sets und Labels in ein CuPy-Array umwandeln.

In [15]:
import cudf
import cupy as cp
import numpy as np
import time
from cuml.svm import SVC
from cuml.metrics import accuracy_score
from sklearn.model_selection import ParameterSampler, ParameterGrid
from cuml.model_selection import GridSearchCV
import joblib

DATAPATH = '../Data/'
MODELPATH = '../Data/Models/SVM/'
MODELPATHSKLEARN = '../Data/Models/SVM/SVM_sklearn/'

train_set = cudf.read_csv(DATAPATH + 'train_set.csv')
train_labels = cudf.read_csv(DATAPATH + 'train_labels.csv')
test_set = cudf.read_csv(DATAPATH + 'test_set.csv')
test_labels = cudf.read_csv(DATAPATH + 'test_labels.csv')


In [18]:
clf = SVC()

clf.fit(train_set, train_labels)

test_pred = clf.predict(test_set)
accuracy_score = accuracy_score(test_labels, test_pred)

joblib.dump(clf, MODELPATH + 'SVM_no_hyper.pkl')

print(f'Accuracy: {accuracy_score}')

Accuracy: 0.7532467246055603


* Ohne Hyperparameter Tuning erhalten wir eine Accuracy von **ca. 0.753** ein wert den wir auf jeden fall noch verbessern wollen.
* Als erstes verwenden wir `ParameterSampler`, um eine breitere Fläche an Hyperparametern testen zu koennen.


In [ ]:
param_distributions = [
    {
        'kernel': ['linear'],
        'C': np.logspace(np.log10(0.001), np.log10(1000), 7, endpoint=True), 
        'tol': np.logspace(np.log10(0.0000001), np.log10(0.1), 7, endpoint=True),
        'max_iter': [1000, 2000, 3000, 4000, 5000],
        'class_weight': ['balanced'],
    },
    {
        'kernel': ['poly'],
        'C': np.logspace(np.log10(0.001), np.log10(1000), 7, endpoint=True), 
        'tol': np.logspace(np.log10(0.0000001), np.log10(0.1), 7, endpoint=True),
        'gamma': np.logspace(np.log10(0.001), np.log10(1000), 7, endpoint=True),
        'degree': [2, 3, 4, 5, 6], # Macht natrülich nur Sinn, wenn degree > 1 da sonst linear
        'max_iter': [1000, 2000, 3000, 4000, 5000],
        'class_weight': ['balanced'],
    },
    {
        'kernel': ['rbf'],
        'C': np.logspace(np.log10(0.001), np.log10(1000), 7, endpoint=True),
        'tol': np.logspace(np.log10(0.0000001), np.log10(0.1), 7, endpoint=True),
        'gamma': np.logspace(np.log10(0.001), np.log10(1000), 7, endpoint=True),
        'max_iter': [1000, 2000, 3000, 4000, 5000],
        'class_weight': ['balanced'],
    },
    {
        'kernel': ['sigmoid'],
        'C': np.logspace(np.log10(0.001), np.log10(1000), 7, endpoint=True),
        'tol': np.logspace(np.log10(0.0000001), np.log10(0.1), 7, endpoint=True),
        'gamma': np.logspace(np.log10(0.001), np.log10(1000), 7, endpoint=True),
        'coef0': np.linspace(-1, 1, 10),
        'max_iter': [1000, 2000, 3000, 4000, 5000],
        'class_weight': ['balanced'],
    }
]

n_iter = 100

best_accuracy = 0
best_params = None

start_time = time.time()

all_sampled_params = []
for params in param_distributions:
    sampler = ParameterSampler(params, n_iter=n_iter, random_state=42)
    sampled_params = list(sampler)
    all_sampled_params.extend(sampled_params)

clf = SVC()

for params in all_sampled_params:
    try:
        clf.set_params(**params)
        clf.fit(train_set, train_labels)
        predictions = clf.predict(test_set)
        accuracy = accuracy_score(test_labels, predictions)

        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_params = params
    except ValueError as e:
        print(f"Skipping invalid parameter combination: {params}")
        print(f"Error: {e}")


end_time = time.time()

best_model = SVC(**best_params)
best_model.fit(train_set, train_labels) 
test_predictions = best_model.predict(test_set)
test_accuracy = accuracy_score(test_labels, test_predictions)

joblib.dump(best_model, MODELPATH + 'SVM_para_sampl.pkl')

['../Data/Models/SVM/SVM_para_sampl.pkl']

In [23]:
print(f"Best parameters: {best_params}")
print(f"Best accuracy: {best_accuracy}")
print(f"Time taken: {end_time - start_time:.2f} seconds")
print(f"Test accuracy with best model: {test_accuracy}")

Best parameters: {'tol': np.float64(0.1), 'max_iter': 4000, 'kernel': 'poly', 'gamma': np.float64(0.1), 'degree': 5, 'class_weight': 'balanced', 'C': np.float64(1.0)}
Best accuracy: 0.7662337422370911
Time taken: 35.68 seconds
Test accuracy with best model: 0.701298713684082


### 5. Parameter-Optimierung mit `ParameterSampler`

Mithilfe des `ParameterSamplers` konnten wir einige vielversprechende Parameter identifizieren:

- Gefundene Parameter:
    -  kernel: poly
    -  C: 1.0
    -  degree: 5
    -  tol: 0.1
    -  gamma: 0.1
    -  max_iter: 4000
    -  class_weight: balanced

- Modell-Performance:
    - Validierungsgenauigkeit: 76.62%
    - Testgenauigkeit: 70.13% (Hinweis auf Overfitting)

- Trainingszeit: 21.55 Sekunden

### 6. Verfeinerung der Hyperparameter mit `ParameterGrid`

Im nächsten Schritt werden wir nun ein `ParameterGrid` verwenden, um die Hyperparameter weiter zu optimieren.

In [25]:
param_grid = {
        'kernel': ['poly'],
        'C': [0.01, 0.1, 0.8, 0.9, 1, 1.1, 1.2, 2, 5, 10], 
        'tol': [0.01, 0.01, 0.1],
        'gamma': [0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.2, 0.3, 0.4, 0.5],
        'degree': [4, 5, 6],
        'max_iter': np.linspace(3900, 4100, 10),
        'class_weight': ['balanced'],
    },

best_accuracy = 0
best_params = None

start_time = time.time()

for params in ParameterGrid(param_grid):
    clf.set_params(**params)
    clf.fit(train_set, train_labels)
    predictions = clf.predict(test_set)
    accuracy = accuracy_score(test_labels, predictions)

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_params = params    

end_time = time.time()

best_model = SVC(**best_params)
best_model.fit(train_set, train_labels)
test_predictions = best_model.predict(test_set)

test_accuracy = accuracy_score(test_labels, test_predictions)

joblib.dump(best_model, MODELPATH + 'SVM_para_grid.pkl')

['../Data/Models/SVM/SVM_para_grid.pkl']

In [26]:
print("\n" + "="*50)
print("GRID SEARCH RESULTS")
print("="*50)
print("\nBest Parameters Found:")
for param, value in best_params.items():
    print(f"  {param}: {value}")

print("\nModel Performance:")
print(f"  Validation Accuracy: {best_accuracy*100:.2f}%")

print(f"  Test Accuracy: {test_accuracy*100:.2f}%")
print(f"\nTraining Time: {end_time - start_time:.2f} seconds")
print("="*50)


GRID SEARCH RESULTS

Best Parameters Found:
  C: 1.2
  class_weight: balanced
  degree: 4
  gamma: 0.07
  kernel: poly
  max_iter: 3900.0
  tol: 0.1

Model Performance:
  Validation Accuracy: 78.57%
  Test Accuracy: 66.88%

Training Time: 436.49 seconds


Obwohl wir scheinbar bessere Parameter gefunden haben, hat sich die Test-Genauigkeit sogar verschlechtert! Ein klares Zeichen für Overfitting.

- Best Parameters Found:
  - C: 1.2
  - class_weight: balanced
  - degree: 4
  - gamma: 0.07
  - kernel: poly
  - max_iter: 3900.0
  - tol: 0.1

- Model Performance:
  - **Validation Accuracy: 78.57%**
  - **Test Accuracy: 66.88%**

- Training Time: 406.26 seconds


# Cross-Validation

Bisher haben wir verschiedene Methoden des Hyperparameter-Tunings betrachtet, um unsere Modelle zu verbessern. Um das volle Potenzial des Tunings auszuschöpfen und vor allem Overfitting zu vermeiden, benötigen wir jedoch ein weiteres wichtiges Werkzeug: die Cross-Validation (Kreuzvalidierung).

Im nächsten Schritt werden wir wieder damit beginnen, die Parameter zufällig zu wählen. Dieses Mal werden wir jedoch zusätzlich die Cross-Validation verwenden.

wir benutzen wieder die selbe Parameter Distrobution wie bei `ParameterSampler`

In [2]:
param_distributions = [
    {
        'kernel': ['linear'],
        'C': np.logspace(np.log10(0.001), np.log10(1000), 7, endpoint=True), 
        'tol': np.logspace(np.log10(0.0000001), np.log10(0.1), 7, endpoint=True),
        'max_iter': [1000, 2000, 3000, 4000, 5000],
        'class_weight': ['balanced'],
    },
    {
        'kernel': ['poly'],
        'C': np.logspace(np.log10(0.001), np.log10(1000), 7, endpoint=True), 
        'tol': np.logspace(np.log10(0.0000001), np.log10(0.1), 7, endpoint=True),
        'gamma': np.logspace(np.log10(0.001), np.log10(1000), 7, endpoint=True),
        'degree': [2, 3, 4, 5, 6], # Macht natrülich nur Sinn, wenn degree > 1 da sonst linear
        'max_iter': [1000, 2000, 3000, 4000, 5000],
        'class_weight': ['balanced'],
    },
    {
        'kernel': ['rbf'],
        'C': np.logspace(np.log10(0.001), np.log10(1000), 7, endpoint=True),
        'tol': np.logspace(np.log10(0.0000001), np.log10(0.1), 7, endpoint=True),
        'gamma': np.logspace(np.log10(0.001), np.log10(1000), 7, endpoint=True),
        'max_iter': [1000, 2000, 3000, 4000, 5000],
        'class_weight': ['balanced'],
    },
    {
        'kernel': ['sigmoid'],
        'C': np.logspace(np.log10(0.001), np.log10(1000), 7, endpoint=True),
        'tol': np.logspace(np.log10(0.0000001), np.log10(0.1), 7, endpoint=True),
        'gamma': np.logspace(np.log10(0.001), np.log10(1000), 7, endpoint=True),
        'coef0': np.linspace(-1, 1, 10),
        'max_iter': [1000, 2000, 3000, 4000, 5000],
        'class_weight': ['balanced'],
    }
]

In [27]:
svm_rnd_cv = RandomizedSearchCV(SVC(), param_distributions, n_iter=50, random_state=42, cv=5)
svm_rnd_cv.fit(train_set, train_labels)

TypeError: Implicit conversion to a host NumPy array via __array__ is not allowed, To explicitly construct a GPU matrix, consider using .to_cupy()
To explicitly construct a host matrix, consider using .to_numpy().